In [ ]:
# initial setup
try:
    # settings colab:
    import google.colab

    ! mkdir -p ../Data
    # los que usan colab deben modificar el token de esta url:
    ! wget -O ../Data/bikes.csv https://raw.githubusercontent.com/Digital-House-DATA/ds_blend_students_2020/master/M3/CLASE_16_Regresion_Lineal_Multiple/Data/bikes.csv?token=AA4GFHIG665I3BPVQCFY3US63APZM
    
except ModuleNotFoundError:    
    # settings local:
    %run "../../../common/0_notebooks_base_setup.py"

[<img src="https://www.digitalhouse.com/ar/logo-DH.png" width="400" height="200" align='right'>](http://digitalhouse.com.ar/)

# PRACTICA INDEPEDIENTE 

In [ ]:
# Leemos los datos y seteamos el datetime como índice.
import numpy as np
import pandas as pd

bikes = pd.read_csv('../Data/bikes.csv', index_col='datetime', parse_dates=True)

In [ ]:
# Dado que "count" es un método de pandas, es conviente renombrar la columna:

bikes.rename(columns={'count':'total'}, inplace=True)

##  Adicionales

### Ingeniería de features

Vea si puede crear las siguientes features:

- ** hora: ** como una sola feature numérica (de 0 a 23)
- ** hora: ** como una feature categórica (use 23 variables dummy)
- ** día: ** como una sola feature categórica (día = 1 de 7am a 8pm y día = 0 de lo contrario)


In [ ]:
# hora como variable numérica
bikes['hora'] = bikes.index.hour

In [ ]:
# hora como categorica

bikes_dummies = pd.get_dummies(bikes.index.hour, drop_first=True)

In [ ]:
bikes_dummies.head()

In [ ]:
bikes_dummies.index = bikes.index

In [ ]:
bikes_dummies.sample(5)

In [ ]:
# días como categórica
a = np.array(((bikes.index.hour) >= 7) & (bikes.index.hour <= 20))

In [ ]:
bikes['dia']= a.astype(int)

In [ ]:
bikes['dia'].sample(5)

In [ ]:
bikes  = pd.concat([bikes,bikes_dummies], axis = 1)

In [ ]:
bikes.head()

##### A continuación, pruebe utilizar cada una de las tres features con `train_test_rmse` para ver cuál funciona mejor!

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
# Definimos una función que acepta una lista de features, hace el split entre train y test,
# reservando un 25% de las observaciones para testeo, y devuelve la prueba RMSE.

from sklearn.model_selection import train_test_split

def train_test_rmse(feature_cols):
    X = bikes[feature_cols]
    y = bikes.total
    # Como estamos trabajando con observaciones ordenadas en el tiempo, ponemos
    # shuffle=False para evitar data leakage
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
    linreg = LinearRegression()
    linreg.fit(X_train, y_train)
    y_pred = linreg.predict(X_test)
    return np.sqrt(metrics.mean_squared_error(y_test, y_pred))

In [ ]:
train_test_rmse(['temp', 'season', 'humidity'])

In [ ]:
train_test_rmse(['temp', 'season', 'humidity','dia'])

In [ ]:
train_test_rmse(['temp', 'season', 'humidity','hora'])

In [ ]:
train_test_rmse(['temp', 'season', 'humidity',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23])

### Crear y comparar modelos con variables cuadráticas

¿Con cuales variables probaría? 

In [ ]:
bikes['temp_2'] = bikes.temp ** 2

In [ ]:
bikes['humidity_2'] = bikes.humidity ** 2

In [ ]:
train_test_rmse(['temp', 'season', 'humidity',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,
               'temp_2', 'humidity_2'])